# Recoding xmls to jsons

In [ ]:
## use com.databricks package specified when spinning up cluster
import os ## restart kernel
#os._exit(00)
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.4.1 pyspark-shell'

## import standard libraries
import re
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

## import pyspark elements
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import col, lit, when
from pyspark.sql import Row

## set checkpoint directory
sc.setCheckpointDir("gs://bucket-brad-project/tmp/")

In [ ]:
## choose datasets to compare
data_array = [
"english",
"maths",
"rus_stackoverflow",
"superuser",
]

In [ ]:
## point to right directory
fpath = 'gs://bucket-brad-project/datasets/large-datasets/'

## function to extract data from xml files
def load_all_xml(table_name):
    # create dataframe list skeleton
    dfs = {}
    xml_name = table_name.capitalize()
    for i in data_array:
        table = sqlContext.read.format('com.databricks.spark.xml').options(rowTag=table_name).load(fpath+i+'.stackexchange.com/'+xml_name+'.xml')
        rdd_table = table.rdd.flatMap(lambda x: x).flatMap(lambda x: x).repartition(96) # key step
        # assign dataframe to list entry
        dfs[i] = sqlContext.createDataFrame(rdd_table, table.schema.fields[0].dataType.elementType)
        dfs[i].createOrReplaceTempView(table_name)
    return dfs

### Load xmls into pyspark dataframes

In [ ]:
## start time
t0 = time.time()
print("Start time:", datetime.now().time())

## extract all datasets and store in list of dataframes
posts_dfs = load_all_xml("posts")

## end time
T = time.time() - t0;
print("Time taken:", round(T/60), "minutes.") 

In [ ]:
# print type of posts_dfs
print(type(posts_dfs))
print('\n')

# print type of first dataframe
print(type(posts_dfs["codereview"]))
print('\n')

# schema of first dataframe
posts_dfs["codereview"].printSchema()

### Write to json

In [ ]:
df_final.coalesce(1).write.format('json').save(data_output_file+"createjson.json", overwrite=True)

In [ ]:
import json
collected_df = df_final.collect()
with open(data_output_file + 'createjson.json', 'w') as outfile:
    json.dump(data, outfile)